In [2]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 892.5 kB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-hj3d0xbm
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-hj3d0xbm
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=f34083009f193db20102247d17c83884383228b6fdb7fa12ffc784545b676126
  Stored in directory: /tmp/pip-ephem-wheel-cache-7hy079vg/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [117]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import clip

# Load CLIP pre-trained weights and set up the processor
# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, clip_preprocess = clip.load('ViT-B/32', device)

In [118]:
for param in clip_model.parameters():
    param.requires_grad = False

In [120]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
# Load the dataset
root = os.path.expanduser("~/.cache")
train = CIFAR10(root, download=True, train=True, transform=clip_preprocess)
test = CIFAR10(root, download=True, train=False, transform=clip_preprocess)

100%|██████████| 170498071/170498071 [00:02<00:00, 77920242.67it/s]


Extracting /root/.cache/cifar-10-python.tar.gz to /root/.cache
Files already downloaded and verified


In [123]:
def get_features(dataset,model):
    all_features = []
    all_labels = []

    with torch.no_grad():
        for images, labels in DataLoader(dataset, batch_size=100):
            features = model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

In [124]:
train_features, train_labels = get_features(train,clip_model)

In [178]:
test_features, test_labels = get_features(test,clip_model)

In [132]:
train_features.shape

(50000, 512)

In [175]:
class Classifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        # Apply softmax activation
        x = nn.functional.softmax(self.fc(x))
        return x

In [176]:
import os
import tqdm
import torch.optim as optim

# Create the classifier
classifier = Classifier(512, 10)
classifier.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

In [171]:
def one_hot_encode(labels, num_classes=10):
    one_hot = []
    for label in labels:
      l = [0]*num_classes
      l[label-1]=1
      one_hot.append(l)

    return one_hot

In [ ]:
train_labels = one_hot_encode(train_labels)
test_labels = one_hot_encode(test_labels)

In [177]:
num_epochs = 5
for epoch in range(num_epochs):
    for i in range(len(train_features)):
        features, labels = torch.Tensor(train_features[i]).to(device), torch.Tensor(train_labels[i]).to(device)

        # Forward pass
        outputs = classifier(features)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

<ipython-input-175-55dda7985d8f>:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(self.fc(x))


Epoch 1/5, Loss: 2.014613389968872
Epoch 2/5, Loss: 2.2185089588165283
Epoch 3/5, Loss: 2.378230333328247
Epoch 4/5, Loss: 2.410783052444458
Epoch 5/5, Loss: 2.3916091918945312


In [197]:
# Evaluate on the test set
classifier.eval()
correct = 0
total = 0

with torch.no_grad():
    for i in range(len(test_features)):
        features, labels = torch.Tensor(test_features[i]).to(device), torch.Tensor(test_labels[i]).to(device)
        outputs = classifier(features)

        predicted = torch.argmax(outputs.data)
        value = torch.argmax(labels)
        total += 1
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

<ipython-input-175-55dda7985d8f>:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(self.fc(x))


Test Accuracy: 95.47%
